In [1]:
#Import Data (lyrics1 lyrics2 similarity_score)
!cp "drive/My Drive/Course/RNN/Final_DATA/final_output.csv" data.csv
import pandas as pd
df = pd.read_csv("data.csv")
df.head()
#df.shape

,lyrics1,lyrics2,similarity_score
0,"['the', 'world', 'is', 'my', 'oyster', 'ha', '...","['oh', 'notheres', 'got', 'to', 'be', 'a', 'be...",1.000000
1,"['ive', 'just', 'erased', 'its', 'been', 'a', ...","['i', 'took', 'a', 'walk', 'in', 'the', 'park'...",1.000000
2,"['ive', 'just', 'erased', 'its', 'been', 'a', ...","['while', 'you', 'skip', 'the', 'night', 'i', ...",0.972941
3,"['light', 'the', 'gasoline', 'ive', 'got', 'th...","['i', 'like', 'it', 'when', 'youre', 'watching...",1.000000
4,"['light', 'the', 'gasoline', 'ive', 'got', 'th...","['when', 'i', 'first', 'met', 'you', 'got', 'a...",0.966398


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!git clone https://github.com/sahilskumar/Music-Recommender-System.git
%cd lstm-siamese-text-similarity
!ls
!pip install -r requirements.txt

fatal: destination path 'lstm-siamese-text-similarity' already exists and is not an empty directory.
/content/lstm-siamese-text-similarity
config.py      images	    inputHandler.py  model.py	requirements.txt
controller.py  __init__.py  LICENSE	     README.md	sample_data.csv


In [8]:
from model import SiameseBiLSTM
from inputHandler import word_embed_meta_data, create_test_data
from config import siamese_config
import pandas as pd

############ Data Preperation ##########

import ast
x1 = list(df['lyrics1'])
x2 = list(df['lyrics2'])
sentences1 = []
sentences2 = []
for i in range(len(x1)):
  listToStr = ' '.join(map(str, ast.literal_eval(x1[i]))) 
  sentences1.append(listToStr)
for i in range(len(x2)):
  listToStr = ' '.join(map(str, ast.literal_eval(x2[i])))
  sentences2.append(listToStr)

is_similar = list(df['similarity_score'])
for i in range(len(list(is_similar))):
  is_similar[i] = 1 if int(is_similar[i]) > 0.5 else 0


Using TensorFlow backend.


In [9]:
######## Word Embedding ############

tokenizer, embedding_matrix = word_embed_meta_data(sentences1 + sentences2,  siamese_config['EMBEDDING_DIM'])

embedding_meta_data = {
	'tokenizer': tokenizer,
	'embedding_matrix': embedding_matrix
}


Embedding matrix shape: (77558, 50)
Null word embeddings: 1


In [10]:
## creating sentence pairs
sentences_pair = [(x1, x2) for x1, x2 in zip(sentences1, sentences2)]
del sentences1
del sentences2
print(len(sentences_pair))

278994


In [11]:
class Configuration(object):
    """Dump stuff here"""

CONFIG = Configuration()

CONFIG.embedding_dim = siamese_config['EMBEDDING_DIM']
CONFIG.max_sequence_length = siamese_config['MAX_SEQUENCE_LENGTH']
CONFIG.number_lstm_units = siamese_config['NUMBER_LSTM']
CONFIG.rate_drop_lstm = siamese_config['RATE_DROP_LSTM']
CONFIG.number_dense_units = siamese_config['NUMBER_DENSE_UNITS']
CONFIG.activation_function = siamese_config['ACTIVATION_FUNCTION']
CONFIG.rate_drop_dense = siamese_config['RATE_DROP_DENSE']
CONFIG.validation_split_ratio = siamese_config['VALIDATION_SPLIT']

siamese = SiameseBiLSTM(CONFIG.embedding_dim , CONFIG.max_sequence_length, CONFIG.number_lstm_units , CONFIG.number_dense_units, 
					    CONFIG.rate_drop_lstm, CONFIG.rate_drop_dense, CONFIG.activation_function, CONFIG.validation_split_ratio)


In [12]:
best_model_path = siamese.train_model(sentences_pair, is_similar, embedding_meta_data, model_save_directory='./')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 251095 samples, validate on 27899 samples



Epoch 1/20
251095/251095 [==============================] - 168s 670us/step - loss: 0.4588 - acc: 0.8256 - val_loss: 0.4437 - val_acc: 0.8296



Epoch 2/20
251095/251095 [==============================] - 167s 665us/step - loss: 0.4419 - acc: 0.8318 - val_loss: 0.4376 - val_acc: 0.8296
Epoch 3/20
251095/251095 [==============================] - 164s 653us/step - loss: 0.4373 - acc: 0.8317 - val_loss: 0.4294 - val_acc: 0.8299
Epoch 4/20
251095/251095 [==============================] - 164s 651us/step - loss: 0.4339 - acc: 0.8316 - val_loss: 0.4285 - val_acc: 0.8295
Epoch 5/20
251095/251095 [==============================] - 165s 656us/step - loss: 0.4304 - acc: 0.8316 - val_loss: 0.4228 - val_acc: 0.8297
Epoch 6/20
251095/251095 [==============================] - 164s 653us/step - loss: 0.4288 - acc: 0.8316 - val_loss: 0.4235 - val_acc: 0.8296
Epoch 7/20
251095/251095 [==============================] - 164s 652us/step - loss: 0.4271 - acc: 0.8316 - val_loss: 0.4211 - val_acc: 0.8295
Epoch 8/20
251095/251095 [==============================] - 166s 660us/step - loss: 0.4253 - acc: 0.8315 - val_loss: 0.4185 - val_acc: 0.8297
Epoch 

In [ ]:
'''
for i in range(len(list(is_similar))):
  is_similar[i] = 1 if int(is_similar[i]) > 0.5 else 0
'''
count = 0
for i in range(len(list(is_similar))):
  count = count+1 if int(is_similar[i]) > 0 else count+0
count

46979

In [3]:
%pycat model.py

In [14]:
from operator import itemgetter
from keras.models import load_model

model = load_model(best_model_path)

test_sentence_pairs = [('What can make Physics easy to learn?','How can you make physics easy to learn?'),('How many times a day do a clocks hands overlap?','What does it mean that every time I look at the clock the numbers are the same?')]

test_data_x1, test_data_x2, leaks_test = create_test_data(tokenizer,test_sentence_pairs,  siamese_config['MAX_SEQUENCE_LENGTH'])

preds = list(model.predict([test_data_x1, test_data_x2, leaks_test], verbose=1).ravel())
results = [(x, y, z) for (x, y), z in zip(test_sentence_pairs, preds)]
results.sort(key=itemgetter(2), reverse=True)
print(results)

2/2 [==============================] - 1s 434ms/step
[('What can make Physics easy to learn?', 'How can you make physics easy to learn?', 0.6160112), ('How many times a day do a clocks hands overlap?', 'What does it mean that every time I look at the clock the numbers are the same?', 0.3182659)]
